In [ ]:
%load_ext autoreload
%autoreload 2

from repo2 import *
import utils as u
import numpy as np
import seaborn as sns
from sklearn.metrics import roc_curve, auc

# FF, GG, DFDC, Ldr

look below for Celeb-DF

In [2]:
# get the file names
train_files, test_files = u.train_test_all_ids('/data/home/shruti/voxceleb/vgg/leaders/')
all_lbls = list(train_files.keys())
all_lbls.sort()
all_lbls = np.array(all_lbls)

before 4105
after 3739


In [3]:

# repo with leaders + ff original + imposters + Google
def get_train_repo(params):
    
    out_repo = Repo2(params['bsfldr'], params['frames'], params['steps'], np.mean, 2000)
    out_repo.add_ids({f: train_files[f] for f in ['bo','br','bs','cb','dt','ew','hc','jb','kh', 'pb',
                                                  'bo_imposter','bs_imposter','ew_imposter','dt_imposter',
                                                   'hc_imposter', 'jb_imposter']}, 0, 1)
    out_repo.add_ids({f: train_files[f] for f in train_files.keys() if 'GG_' in f}, 0, 1)
    out_repo.add_ids({k: train_files[k] for k in train_files.keys() if 'DFDC_' in k}, 0, 1)
    out_repo.add_ids({f: train_files[f] for f in train_files.keys() if 'FF_' in f}, 0, 0.5)    
    out_repo.add_ids({f: train_files[f] for f in train_files.keys() if 'CDF_' in f}, 0, 0.5)
    out_repo.add_ids({f: train_files[f] for f in train_files.keys() if 'inwild_' in f}, 0, 1)
    
    
    #out_repo.add_ids({'jen_l': train_files['jen_l']}, 0, 1)
    
    out_repo.build_repo_noKDD()
    
    return out_repo


def save_results(vgg_repo, fab_repo, test_ids, r_or_f, N_f, out_file, n1, n2, parallel=False):
        
    # number of files per id
    test_dict = {}
    for ids in test_ids:
        cur_test_files = test_files[r_or_f][ids]
        if N_f>0 and len(cur_test_files)>N_f:
            test_dict[ids] = cur_test_files[:N_f].copy()
        else:
            test_dict[ids] = cur_test_files.copy()
            
    vgg_result = vgg_repo.dist_using_dict(test_dict, n1, n2, parallel=parallel)
    vgg_result['RealFake'] = r_or_f
    fab_result = fab_repo.dist_using_dict(test_dict, n1, n2, parallel=parallel)
    fab_result['RealFake'] = r_or_f
    full_df = vgg_result.set_index('fileName').join(fab_result.set_index('fileName'), 
                                                 how='inner', lsuffix='_vgg', rsuffix='_fab')
    full_df = full_df[['actualLabel_vgg', 
                       'RealFake_vgg', 'sim_vgg', 'sim_fab']].copy()
    
    if out_file is not None:
        full_df.to_pickle(out_file)
        
    return full_df

# Metric Learning

### Repo of Leaders + Face Forensics Original + Imposter + Google

In [4]:

vgg_repo_metric = get_train_repo({'istrain':True, 'bsfldr':'/data/home/shruti/voxceleb/vgg/leaders/', 
                                'frames':100, 'steps':5, 'N_comp':512})
fab_repo_metric = get_train_repo({'istrain':True, 'bsfldr':'/data/home/shruti/voxceleb/fabnet_metric/', 
                                'frames':1, 'steps':1, 'N_comp':-1})

# this id is repeating with CDF_id12
#vgg_repo_metric.remove_ids(['jen_l'])
#fab_repo_metric.remove_ids(['jen_l'])

u.save_obj(vgg_repo_metric, 'result_csv', 'vgg_repo')
u.save_obj(fab_repo_metric, 'result_csv', 'fab_repo')


Number of labels 1173
Build repo time: 10.325 Size: (176137, 4096)
/data/home/shruti/voxceleb/fabnet_metric/celeb_real/_id27_0005.npy
Number of labels 1173
Build repo time: 2.850 Size: (175081, 512)


AttributeError: 'Repo2' object has no attribute 'label_dict'

In [10]:

u.save_obj(fab_repo_metric.name2label_dict, 'result_csv', 'name2label_dict')


In [6]:

#inwild sb
temp = save_results(vgg_repo_metric, fab_repo_metric, 
             ['CDF_id12'], 
             'real', -1, 
             None, 0, 1, parallel=True)
temp.to_pickle('result_csv/inwild_jl_real_100.csv')

#inwild tc
temp = save_results(vgg_repo_metric, fab_repo_metric, 
             ['inwild_bh'], 
             'real', -1, 
             None, 0, 1, parallel=True)
temp.to_pickle('result_csv/inwild_bh_real_100.csv')

# inwild be
temp = save_results(vgg_repo_metric, fab_repo_metric, 
             ['inwild_an'], 
             'real', -1, 
             None, 0, 1, parallel=True)
temp.to_pickle('result_csv/inwild_an_real_100.csv')



[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done   9 out of   9 | elapsed:   28.8s finished
[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done   9 out of   9 | elapsed:   10.9s finished
[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done   5 out of   5 | elapsed:    8.1s finished
[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done   5 out of   5 | elapsed:    3.8s finished
[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done   1 out of   1 | elapsed:    8.9s finished
[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done   1 out of   1 | elapsed:    3.8s finished


In [7]:

#inwild sb
temp = save_results(vgg_repo_metric, fab_repo_metric, 
             ['inwild_sb'], 
             'fake', -1, 
             None, 0, 1, parallel=True)
temp.to_pickle('result_csv/inwild_sb_fake_100.csv')

#inwild tc
temp = save_results(vgg_repo_metric, fab_repo_metric, 
             ['inwild_tc'], 
             'fake', -1, 
             None, 0, 1, parallel=True)
temp.to_pickle('result_csv/inwild_tc_fake_100.csv')

# inwild be
temp = save_results(vgg_repo_metric, fab_repo_metric, 
             ['inwild_be'], 
             'fake', -1, 
             None, 0, 0.5, parallel=True)
temp.to_pickle('result_csv/inwild_be_fake_100.csv')



[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done   1 out of   1 | elapsed:   12.1s finished
[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done   1 out of   1 | elapsed:    5.1s finished
[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done   1 out of   1 | elapsed:    8.7s finished
[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done   1 out of   1 | elapsed:    3.8s finished
[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done   1 out of   1 | elapsed:    8.8s finished
[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done   1 out of   1 | elapsed:    3.6s finished


In [8]:
vgg_repo_metric.bs_fldr = '/data/home/shruti/voxceleb/vgg/leaders/'
fab_repo_metric.bs_fldr = '/data/home/shruti/voxceleb/fabnet_metric/'


#DFDC
temp = save_results(vgg_repo_metric, fab_repo_metric, 
             [k for k in test_files['real'].keys() if 'DFDC_' in k], 
             'real', -1, 
             None, 0, 1, parallel=True)
temp.to_pickle('result_csv/DFDC_real_100.csv')
temp = save_results(vgg_repo_metric, fab_repo_metric, 
             [k for k in test_files['fake'].keys() if 'DFDC_' in k], 
             'fake', -1, 
             None, 0, 1, parallel=True)
temp.to_pickle('result_csv/DFDC_fake_100.csv')

# GG
temp = save_results(vgg_repo_metric, fab_repo_metric, 
             [k for k in test_files['real'].keys() if 'GG_' in k], 
             'real', -1, 
             None, 0, 1, parallel=True)
temp.to_pickle('result_csv/GG_real_100.csv')
temp = save_results(vgg_repo_metric, fab_repo_metric, 
             [k for k in test_files['fake'].keys() if 'GG_' in k], 
             'fake', -1, 
             None, 0, 1, parallel=True)
temp.to_pickle('result_csv/GG_fake_100.csv')




[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:   32.5s
[Parallel(n_jobs=16)]: Done 168 tasks      | elapsed:  2.6min
[Parallel(n_jobs=16)]: Done 247 out of 247 | elapsed:  3.7min finished
[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:   12.4s
[Parallel(n_jobs=16)]: Done 168 tasks      | elapsed:  1.1min
[Parallel(n_jobs=16)]: Done 247 out of 247 | elapsed:  1.6min finished
[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:   30.3s
[Parallel(n_jobs=16)]: Done 168 tasks      | elapsed:  2.6min
[Parallel(n_jobs=16)]: Done 418 tasks      | elapsed:  6.0min
[Parallel(n_jobs=16)]: Done 768 tasks      | elapsed: 11.1min
[Parallel(n_jobs=16)]: Done 1218 tasks      | elapsed: 17.2min
[Parallel(n_jobs=16)]: Done 1768 tasks      | elapsed: 25.0min
[Para

In [ ]:


#LEADER
temp = save_results(vgg_repo_metric, fab_repo_metric, 
             ['bo','bs','dt','ew','hc','jb'], 
             'real', 10, 
             None, 0, 1, parallel=True)
temp.to_pickle('result_csv/ldr_real_100.csv')
temp = save_results(vgg_repo_metric, fab_repo_metric, 
             ['bo','bs','dt','ew','hc','jb'], 
             'fake', 10, 
             None, 0, 1, parallel=True)
temp.to_pickle('result_csv/ldr_fake_100.csv')


# FF
temp = save_results(vgg_repo_metric, fab_repo_metric, 
             [k for k in test_files['real'].keys() if 'FF_' in k], 
             'real', -1, 
             None, 0.5, 1, parallel=True)
temp.to_pickle('result_csv/FF_real_100.csv')
temp = save_results(vgg_repo_metric, fab_repo_metric, 
             [k for k in test_files['fake'].keys() if 'FF_' in k], 
             'fake', -1, 
             None, 0.5, 1, parallel=True)
temp.to_pickle('result_csv/FF_fake_100.csv')


In [11]:

#Celeb-DF
temp = save_results(vgg_repo_metric, fab_repo_metric, 
             [k for k in test_files['real'].keys() if 'CDF_' in k], 
             'real', -1, 
             None, 0.5, 1, parallel=True)
temp.to_pickle('result_csv/CDF_real_100.csv')
temp = save_results(vgg_repo_metric, fab_repo_metric, 
             [k for k in test_files['fake'].keys() if 'CDF_' in k], 
             'fake', -1, 
             None, 0.5, 1, parallel=True)
temp.to_pickle('result_csv/CDF_fake_100.csv')


[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:   38.4s
[Parallel(n_jobs=16)]: Done 168 tasks      | elapsed:  2.1min
[Parallel(n_jobs=16)]: Done 418 tasks      | elapsed:  4.7min
[Parallel(n_jobs=16)]: Done 591 out of 591 | elapsed:  6.6min finished
[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    8.7s
[Parallel(n_jobs=16)]: Done 168 tasks      | elapsed:   49.8s
[Parallel(n_jobs=16)]: Done 418 tasks      | elapsed:  2.0min
[Parallel(n_jobs=16)]: Done 591 out of 591 | elapsed:  2.7min finished
[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:   23.2s
[Parallel(n_jobs=16)]: Done 168 tasks      | elapsed:  1.9min
[Parallel(n_jobs=16)]: Done 418 tasks      | elapsed:  4.5min
[Parallel(n_jobs=16)]: Done 768 tasks      | elapsed:  8.3min
[Parall

# random code

In [ ]:
vgg_repo_metric = u.load_obj('result_csv', 'vgg_repo')
fab_repo_metric = u.load_obj('result_csv', 'fab_repo')

In [ ]:
vgg_repo_metric.remove_ids(['steve_b'])
fab_repo_metric.remove_ids(['steve_b'])

In [ ]:
# add in wild ids 
vgg_repo_metric.add_ids({f: train_files[f] for f in train_files.keys() if 'inwild_' in f}, 0, 1)
vgg_repo_metric.build_repo_noKDD()

fab_repo_metric.add_ids({f: train_files[f] for f in train_files.keys() if 'inwild_' in f}, 0, 1)
fab_repo_metric.build_repo_noKDD()


In [ ]:
fab_repo_metric.label_dict

In [ ]:
np.save('result_csv/label_dict.npy', fab_repo_metric.label_dict)

# compression

In [ ]:
vgg_repo_metric.bs_fldr = '/data/home/shruti/voxceleb/vgg/compression/'
fab_repo_metric.bs_fldr = '/data/home/shruti/voxceleb/fabnet_metric_compression/'


#LEADER
temp = save_results(vgg_repo_metric, fab_repo_metric, 
             ['bo','bs','dt','ew','hc','jb'], 
             'real', 10, 
             None, 0, 1, parallel=True)
temp.to_pickle('result_csv/ldr_real_100_onlycomp.csv')
temp = save_results(vgg_repo_metric, fab_repo_metric, 
             ['bo','bs','dt','ew','hc','jb'], 
             'fake', 10, 
             None, 0, 1, parallel=True)
temp.to_pickle('result_csv/ldr_fake_100_onlycomp.csv')

#DFDC
temp = save_results(vgg_repo_metric, fab_repo_metric, 
             [k for k in test_files['real'].keys() if 'DFDC_' in k], 
             'real', -1, 
             None, 0., 1, parallel=True)
temp.to_pickle('result_csv/DFDC_real_100_onlycomp.csv')


temp = save_results(vgg_repo_metric, fab_repo_metric, 
             [k for k in test_files['fake'].keys() if 'DFDC_' in k], 
             'fake', -1, 
             None, 0., 1, parallel=True)
temp.to_pickle('result_csv/DFDC_fake_100_onlycomp.csv')

# FF
temp = save_results(vgg_repo_metric, fab_repo_metric, 
             [k for k in test_files['real'].keys() if 'FF_' in k], 
             'real', -1, 
             None, 0.5, 1, parallel=True)
temp.to_pickle('result_csv/FF_real_100_onlycomp.csv')
temp = save_results(vgg_repo_metric, fab_repo_metric, 
             [k for k in test_files['fake'].keys() if 'FF_' in k], 
             'fake', -1, 
             None, 0.5, 1, parallel=True)
temp.to_pickle('result_csv/FF_fake_100_onlycomp.csv')


# GG
temp = save_results(vgg_repo_metric, fab_repo_metric, 
             [k for k in test_files['real'].keys() if 'GG_' in k], 
             'real', -1, 
             None, 0., 1, parallel=True)
temp.to_pickle('result_csv/GG_real_100_onlycomp.csv')
temp = save_results(vgg_repo_metric, fab_repo_metric, 
             [k for k in test_files['fake'].keys() if 'GG_' in k], 
             'fake', -1, 
             None, 0., 1, parallel=True)
temp.to_pickle('result_csv/GG_fake_100_onlycomp.csv')

#Celeb-DF
temp = save_results(vgg_repo_metric, fab_repo_metric, 
             [k for k in test_files['real'].keys() if 'CDF_' in k], 
             'real', -1, 
             None, 0.5, 1, parallel=True)
temp.to_pickle('result_csv/CDF_real_100_onlycomp.csv')
temp = save_results(vgg_repo_metric, fab_repo_metric, 
             [k for k in test_files['fake'].keys() if 'CDF_' in k], 
             'fake', -1, 
             None, 0.5, 1, parallel=True)
temp.to_pickle('result_csv/CDF_fake_100_onlycomp.csv')


[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:  1.0min
[Parallel(n_jobs=16)]: Done  60 out of  60 | elapsed:  1.6min finished
[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:   14.6s
[Parallel(n_jobs=16)]: Done  60 out of  60 | elapsed:   28.5s finished
[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:   31.5s
[Parallel(n_jobs=16)]: Done  57 out of  57 | elapsed:   55.7s finished
[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:   12.0s
[Parallel(n_jobs=16)]: Done  57 out of  57 | elapsed:   22.3s finished
[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:   22.0s
[Parallel(n_jobs=16)]

# Celeb-DF, GG, DFDC, Ldr, FF

In [ ]:

vgg_repo_metric.bs_fldr = '/data/home/shruti/voxceleb/vgg/compression/'
fab_repo_metric.bs_fldr = '/data/home/shruti/voxceleb/fabnet_metric_compression/'


#LEADER
temp = save_results(vgg_repo_metric, fab_repo_metric, 
             ['bo','bs','dt','ew','hc','jb'], 
             'real', 10, 
             None, 0, 1, parallel=True)
temp.to_pickle('result_csv/ldr_real_100_onlycomp.csv')
temp = save_results(vgg_repo_metric, fab_repo_metric, 
             ['bo','bs','dt','ew','hc','jb'], 
             'fake', 10, 
             None, 0, 1, parallel=True)
temp.to_pickle('result_csv/ldr_fake_100_onlycomp.csv')

#DFDC
temp = save_results(vgg_repo_metric, fab_repo_metric, 
             [k for k in test_files['real'].keys() if 'DFDC_' in k], 
             'real', -1, 
             None, 0., 1, parallel=True)
temp.to_pickle('result_csv/DFDC_real_100_onlycomp.csv')


temp = save_results(vgg_repo_metric, fab_repo_metric, 
             [k for k in test_files['fake'].keys() if 'DFDC_' in k], 
             'fake', -1, 
             None, 0., 1, parallel=True)
temp.to_pickle('result_csv/DFDC_fake_100_onlycomp.csv')

# FF
temp = save_results(vgg_repo_metric, fab_repo_metric, 
             [k for k in test_files['real'].keys() if 'FF_' in k], 
             'real', -1, 
             None, 0.5, 1, parallel=True)
temp.to_pickle('result_csv/FF_real_100_onlycomp.csv')
temp = save_results(vgg_repo_metric, fab_repo_metric, 
             [k for k in test_files['fake'].keys() if 'FF_' in k], 
             'fake', -1, 
             None, 0.5, 1, parallel=True)
temp.to_pickle('result_csv/FF_fake_100_onlycomp.csv')


# GG
temp = save_results(vgg_repo_metric, fab_repo_metric, 
             [k for k in test_files['real'].keys() if 'GG_' in k], 
             'real', -1, 
             None, 0., 1, parallel=True)
temp.to_pickle('result_csv/GG_real_100_onlycomp.csv')
temp = save_results(vgg_repo_metric, fab_repo_metric, 
             [k for k in test_files['fake'].keys() if 'GG_' in k], 
             'fake', -1, 
             None, 0., 1, parallel=True)
temp.to_pickle('result_csv/GG_fake_100_onlycomp.csv')

#Celeb-DF
temp = save_results(vgg_repo_metric, fab_repo_metric, 
             [k for k in test_files['real'].keys() if 'CDF_' in k], 
             'real', -1, 
             None, 0.5, 1, parallel=True)
temp.to_pickle('result_csv/CDF_real_100_onlycomp.csv')
temp = save_results(vgg_repo_metric, fab_repo_metric, 
             [k for k in test_files['fake'].keys() if 'CDF_' in k], 
             'fake', -1, 
             None, 0.5, 1, parallel=True)
temp.to_pickle('result_csv/CDF_fake_100_onlycomp.csv')


